In [ ]:
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.overwriteoutput = True

env.workspace = "F:\DriversFireProject\TEMP"

In [ ]:
import requests
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd
import urllib
import time
from datetime import datetime, date, time, timedelta
import io
import requests
import zipfile
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
def shpFiles(rootPath, ext):
    emptyList = []
    root = rootPath
    for path, subdirs, files in os.walk(root):
        for names in files: 
            if names.endswith(ext) and not names.startswith("._"):
                emptyList.append(path + '\\' + names)
    return(emptyList)

In [ ]:
ignitionFiles = shpFiles(r'F:\DriversFireProject\Ignition\Centroid', '.shp')
USFSIgnition = r'F:\DriversFireProject\Ignition\USFS_Ignitions\Ignition12_15.shp'
dateCSV = pd.read_csv(r'F:\DriversFireProject\NaturalNeighborResults\FinalDF.csv', index_col=0)
dateCSV.head()

In [ ]:
# get start dates 
idx = dateCSV.groupby(['Fire', 'Year'])['JD_B'].transform(min) == dateCSV['JD_B']
firstDay = dateCSV[idx]
firstDay

In [ ]:
# Add attributes: FIRE_NAME, FIRE_YEAR, DISCOVERY_ (datetime) 

fire = [] 
year = [] 
julianDay = [] 
xcoord = [] 
ycoord = [] 

for ign in ignitionFiles: 
    nm = ign.split("\\")
    name = nm[-1]
    nmm = name.split("_")
    fr = nmm[0]
    yr = nmm[1]
    try:
        jd = firstDay.loc[(firstDay.Fire == fr) & (firstDay.Year == int(yr)), 'JD_B'].item()
        with arcpy.da.SearchCursor(ign, ['POINT_X', 'POINT_Y',]) as cursor:
            for row in cursor:
                xcoord.append(row[0])
                ycoord.append(row[1])
                fire.append(fr) 
                year.append(yr) 
                julianDay.append(jd)
                print(yr, fr, jd, row[0], row[1])
    except: 
        print("error", fr, yr)
        continue
            
ignitionDF = pd.DataFrame({'Fire': fire, 'Year': year,'JulianDay': julianDay, 
                       'X': xcoord, 'Y': ycoord})
ignitionDF

In [ ]:
ignitionDF.to_csv(r"F:\DriversFireProject\Ignition\ignitionDF.csv")

In [ ]:
# Process: Convert Table To CSV File
arcpy.TableToTable_conversion(USFSIgnition, r"F:\DriversFireProject\Ignition", "USFSIgnitions.csv")

In [ ]:
USFS_CSV = pd.read_csv(r'F:\DriversFireProject\Ignition\USFSIgnitions.csv', index_col=0)
USFS_CSV

In [ ]:
USFS_CSV.dtypes

In [ ]:
import difflib

In [ ]:
# pip install fuzzywuzzy
from fuzzywuzzy import fuzz

def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [ ]:
ignitionDF['Year'].astype('int64')
ignitionDF = ignitionDF[ignitionDF['Year'].astype('int64') < 2016]

In [ ]:
# List for dicts for easy dataframe creation
dict_list = []
# iterating over our players without salaries found above
for name in ignitionDF.Fire:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, listUSFS, 70)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"FireName" : name})
    dict_.update({"USFS Name" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table

In [ ]:
listUSFS = USFS_CSV.FIRE_NAME.tolist()
listUSFS

listUSFS = [x.capitalize() for x in listUSFS]
listUSFS
USFS_CSV['Fire']= listUSFS

In [ ]:
listUSFS[listUSFS[]]

In [ ]:
NonMatch = merge_table[merge_table['score'] == -1]
NonMatchList = NonMatch.FireName.tolist()
NonMatchList

In [ ]:
newNames = ['Coffee', 'Fryingpan Faulkstein','Steelhead', 'Md Flake', 'Clear', 'Buck', 'Pelletreau']

In [ ]:
newNames = ['combat',
 'pulgas',
 'tomahawk',
 'corral',
 'barker',
 'blue',
 'peak',
 'rail',
 'shiell',
 'bear',
 'coon',
 'peak',
 'fryingpan faulkstein',
 'fryingpan',
 'log',
 'man',
 'whites',
 'gobbler',
 'lassics',
 'pickett',
 'gulch',
 'johnson',
 'boulder',
 'shelly',
 'cocos']

newNames = [x.title() for x in newNames]
newNames

In [ ]:
ignitionDF = ignitionDF.replace(NonMatchList, newNames)

In [ ]:
ignitionDF

In [ ]:
ignitionDF

In [ ]:
am = USFS_CSV[USFS_CSV['Fire'] =='American']
am['DISCOVERY1']

In [ ]:
ignitionDF['JulianDay'] = ignitionDF['JulianDay'].astype('int64')
ignitionDF['Year'] = ignitionDF['Year'].astype('int64')
mergedIgnition = pd.merge(ignitionDF, USFS_CSV, how='left', left_on=['Fire', 'Year'], right_on=['Fire', 'FIRE_YEAR'])

In [ ]:
len(mergedIgnition['Fire'].unique())

In [ ]:
mergedIgnition.to_csv(r'F:\DriversFireProject\Ignition\MergedIgnitions.csv')

In [ ]:
mergedIgnition['dif'] = (mergedIgnition['JulianDay'] - mergedIgnition['DISCOVERY1']).abs()
mergedIgnition['is_closest'] = mergedIgnition.groupby(['Fire', 'Year'])['dif'].transform('min') == mergedIgnition['dif']
mergedIgnition = mergedIgnition[mergedIgnition['is_closest'] == True]
mergedIgnition.to_csv(r'F:\DriversFireProject\Ignition\MergedIgnitions_FILT.csv')

In [ ]:
mergedIgnition

In [ ]:
mergedIgnition = mergedIgnition.sort_values(["JulianDay"], ascending = True)
mergedIgnition['INT_XY'] = list(zip(mergedIgnition.Y, mergedIgnition.X))
mergedIgnition['USFS_XY'] = list(zip(mergedIgnition.POINT_Y, mergedIgnition.POINT_X))

In [ ]:
mergedIgnition

In [ ]:
import math
from math import radians, degrees, sin, cos, asin, acos, sqrt
def calculateDistance(pointA, pointB):
    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])
    lon1 = math.radians(pointA[1])
    lon2 = math.radians(pointB[1])
    return 6371 * (
        acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2))
    )

In [ ]:
calculateDistance((34.1563783018, -117.862617368), (40.1900947981, -121.597906602))

In [ ]:
INTList = mergedIgnition.INT_XY.tolist()
USFSList = mergedIgnition.USFS_XY.tolist()

dist = [] 

for f, b in zip(INTList, USFSList):
    dist.append(calculateDistance(f, b))

mergedIgnition['Distance'] = dist
mergedIgnition

In [ ]:
import seaborn as sns
sns.set(style="white")

cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)
ax = sns.scatterplot(x="X", y="Y",
                     hue="Year", size="Distance",
                     palette=cmap, sizes=(10, 200),
                     data=mergedIgnition)

In [ ]:
mergedIgnition['dif'].max()

In [ ]:
mergedIgnition = mergedIgnition.sort_values(by=['Distance'])
mergedIgnition['Fire'].tolist()

In [ ]:
distList = mergedIgnition['Distance'].tolist()
sorted(distList)


In [ ]:
sample.groupby(['Distance'])['Distance'].count()